In [3]:
# Only do this install once, for experimenting with hugging face's transformers
# CPU usage, just to ensure that model architecture will be working.
# Will later likely need to use GPUs using a virtual environment,
# IF: our dataset is too big, possibly not the case.
#pip install transformers[torch]

# Same here, run once
#!pip install datasets

   ---------------------------------------- 0.0/542.0 kB ? eta -:--:--
   ---------------------- ---------------- 317.4/542.0 kB 19.2 MB/s eta 0:00:01
   ---------------------------------------- 542.0/542.0 kB 8.4 MB/s eta 0:00:00
   ---------------------------------------- 0.0/116.3 kB ? eta -:--:--
   ---------------------------------------- 116.3/116.3 kB 6.6 MB/s eta 0:00:00
   ---------------------------------------- 0.0/371.6 kB ? eta -:--:--
   -------------------------------- ------ 307.2/371.6 kB 18.6 MB/s eta 0:00:01
   ---------------------------------------- 371.6/371.6 kB 5.8 MB/s eta 0:00:00
   ---------------------------------------- 0.0/25.9 MB ? eta -:--:--
   - -------------------------------------- 0.7/25.9 MB 22.4 MB/s eta 0:00:02
   - -------------------------------------- 1.2/25.9 MB 14.7 MB/s eta 0:00:02
   - -------------------------------------- 1.2/25.9 MB 14.7 MB/s eta 0:00:02
   - -------------------------------------- 1.2/25.9 MB 14.7 MB/s eta 0:00:02
   - 

In [7]:
# imports need necessary installs found above

from naiveModel import NBLangIDModel
from BERTModel import BERTGenreClassification, train_model
import pandas as pd
from sklearn.model_selection import train_test_split
from util import get_dataloader

# don't need cuda until using virtual machine
from torch import manual_seed#, cuda

In [8]:
%load_ext autoreload
%autoreload 2

In [6]:
naiveBayes = NBLangIDModel()

c:\Users\oscar\anaconda3\envs\ml-0451\lib\site-packages\huggingface_hub\file_download.py:148: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\oscar\.cache\huggingface\hub\models--distilbert-base-uncased. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to see activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)


In [9]:
BERT = BERTGenreClassification()

In [10]:
# initialize model and dataloaders

# change later, if using virtual machine
device ='cpu' #"cuda" if cuda.is_available() else "cpu"

# seed the model before initializing weights so that your code is deterministic
manual_seed(457)

freeze_bert = False # change later
batch_size = 16 # default
epochs = 5 # default
learning_rate = 1e-2 # default

model = BERTGenreClassification(freeze_bert = freeze_bert).to(device)

# have to change these to implement from df, not from csv
train_dataloader = get_dataloader("train", batch_size=batch_size)
dev_dataloader = get_dataloader("dev", batch_size=batch_size)


# currently, embedding dimensions are wrong
#train_model(model, train_dataloader, dev_dataloader, epochs, learning_rate)

FileNotFoundError: [Errno 2] No such file or directory: '/home/lbiester/CS457HW/HW5/data/train.tsv'

In [4]:
# load data, train test split
descriptions = pd.read_csv("cleanedData.csv")

print("Shape before dropping NaN values:", descriptions.shape)
descriptions = descriptions.dropna()
print("Shape after dropping NaN values:", descriptions.shape)

#TESTING THIS
# print("Rows to be dropped:")
# print(descriptions[descriptions['description'].isna() | ~descriptions['description'].apply(lambda x: isinstance(x, str))])
# #drop the 511 rows where description column is NaN or not a string
# descriptions = descriptions.dropna(subset=['description'])
# descriptions = descriptions[descriptions['description'].apply(lambda x: isinstance(x, str))]

train, test = train_test_split(descriptions, test_size=0.2)

Shape before dropping NaN values: (42661, 5)
Shape after dropping NaN values: (37478, 5)


In [5]:
train.drop("Unnamed: 0", axis= 1)
test.drop("Unnamed: 0", axis= 1)

,description,genre1,genre2,genre3
6202,The WINGS OF FIRE saga continues!Deep in the r...,'Fantasy','Dragons','Middle Grade'
28603,A summertime story only Elin Hilderbrand can t...,'Fiction','Chick Lit','Romance'
9532,The Wild Ass's Skin is Honoré de Balzac's 1831...,'Classics','France','Fiction'
39280,"Astutely observed and deftly witty, One Perfec...",'Nonfiction','Sociology','Marriage'
16179,A murder story with a devilish twist.Nick want...,'Horror','Young Adult','Fantasy'
...,...,...,...,...
3733,Dead Wrong is a study of the scientific and fo...,'Nonfiction','True Crime','History'
24405,"""I have written about the joys of love. I have...",'Historical Fiction','Fiction','Historical'
38663,Caldecott Honor BookWhat happens when Sal and ...,'Picture Books','Childrens','Fiction'
21685,Terra Cotta Warriors have been discovered on o...,'Science Fiction','Young Adult','Fantasy'


In [6]:
train_X = train['description']
train_y1 = train['genre1']
train_y2 = train['genre2']
train_y3 = train['genre3']

In [7]:
# fit the NB model 
naiveBayes.fit(train_X.tolist(), train_y1.tolist())